# TP3: Entrenando un modelo de clasificación

FECHA DE ENTREGA: 16/09

Para este trabajo práctico nos vamos a enfocar en el entrenamiento y evaluación de modelos de clasificación. La novedad es que vamos a incluir un set de datos nuevo. Ya está subida al repositorio una serie de posteos de r/espanol, un foro sin un carácter específico que nuclea conversaciones random en español. La idea es que entrenemos un modelo que clasifique los textos según su pertenencia a los tres subreddits que ahora forman parte de nuestros datos. 

Les dejo una lista de tareas para organizar el tp:

1. Preparación de los Datos

    Revisión del Preprocesamiento: Vienen probando distintas técnicas de preprocesado. Pueden trabajar en limpiar el código, modularizarlo y documentarlo apropiadamente. De esta forma va a ser más fácil implementar pequeñas variaciones a los datos para comparar rendimientos de clasificación y también aplicar los procesamientos ya desarrollados a los nuevos datos de r/espanol.

    Si bien los diferentes modelos y sus hiperparámetros se llevan siempre la atención, la calidad del preprocesamiento va a impactar directamente en el rendimiento de los modelos de clasificación.

    Vectorización: Esto también ya lo hemos incluido en los trabajos anteriores, ahora pueden comparar rendimientos entre una vectorización con **CountVectorizer** y una con **TfidfVectorizer**. Recuerden que los vectorizadores también tienen parámetros seteables, un ejemplo puede ser **ngram_range**  

2. Entrenamiento de Modelos

    Probar Diversos Modelos: Experimenten con al menos tres modelos de clasificación y utilicen técnicas como **GridSearchCV** para encontrar la mejor combinación de hiperparámetros para los modelos que elijan. Aunque el objetivo final es un modelo de clasificación que distinga entre los tres subreddits, si lo consideran útil también pueden comenzar entrenando uno sólo para los dos foros originales.

3. Evaluación del Modelo

    Métricas de Evaluación: Miden el rendimiento de los modelos utilizando métricas clave, tales como los informes de clasificación y las matrices de confusión.

    Comparación de Resultados: Comparen el desempeño de los diferentes modelos y técnicas de vectorización. Analicen qué combinaciones ofrecieron mejores resultados y discutan las posibles razones.

Una sugerencia que puede ser interesante en esta etapa es utilizar una **Pipeline** para organizar y simplificar el proceso de entrenamiento de modelos. Con esta forma de organizar el código podemos encadenar múltiples etapas de procesamiento y modelado en una única estructura, asegurando que las transformaciones se apliquen de manera consistente tanto en el entrenamiento como en la evaluación.

4. Es importante ahora que puedan interpretar los resultados obtenidos. Por ejemplo: ¿qué significan los valores de accuracy o recall que obtuvieron para cada categoría? ¿Con alguna tuvieron peores resultados que con otra? ¿Existieron muchos datos mal clasificados entre ansiedad y depresión? ¿Cómo se comportó el modelo en relación a r/espanol?

5. Una tarea interesante puede ser la exploración de los datos mal clasificados. Observan alguna regularidad entre los comentarios mal clasificados. Pregunta ejemplo: ¿qué palabras son más comunes entre los datos que son de r/ansiedad pero fueron clasificados por nuestro modelo como de r/depresion? ¿Se referían quizás a padecimientos similares? ¿Los comentarios de agradecimiento fueron difíciles de clasificar? 

¿Qué tipo de posteos de r/espanol nuestro modelo está clasificando como propios de r/ansiedad o r/Depresion?


Si bien no es necesario que desarrollen esto ahora, creo que de cara al video final puede ser interesante que piensen en las siguientes preguntas

¿Qué obstáculos encontraron a lo largo de los tres trabajos?
¿Cómo creen que podría mejorarse el rendimiento obtenido en los diferentes modelos?
¿Qué tipo de implementación creen que podría tener un trabajo como este?



## Notebook demo
Acá abajo les dejo un preprocesado simple, seguido de una pipeline que prueba dos tipos de vectorizadores, y testea hiperparámetros para un solo modelo.

In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/patriciof3/mentoria13_famaf_2024/main/df_reddit_mentoria.csv"

df = pd.read_csv(url)

In [2]:
url_espanol = "https://raw.githubusercontent.com/patriciof3/mentoria13_famaf_2024/main/df_espanol_posts.csv"

df_espanol = pd.read_csv(url_espanol)

df_espanol = df_espanol.sample(n=1600, random_state=1) #sampleo de 600 casos para tener un número similar al trabajado previamente

In [3]:
df_merged = pd.concat([df, df_espanol], ignore_index=True)

In [4]:
df_merged.subreddit.value_counts()

Depresion    1619
espanol      1600
ansiedad     1528
Name: subreddit, dtype: int64

### Preprocesado

In [5]:
import pandas as pd
import re
import spacy
from sklearn.model_selection import train_test_split

# Cargar el modelo de Spacy
nlp = spacy.load("es_core_news_sm")
stop_words = spacy.lang.es.stop_words.STOP_WORDS

def preprocess_text(texto):
    texto = texto.drop_duplicates()
    texto = texto.str.lower()
    texto = texto[texto.str.len() >= 30]
    texto = texto.apply(lambda x: re.sub(r'[^a-zA-Z\sáéíóúüñÁÉÍÓÚÜÑ]', '', x))
    texto = texto[~texto.str.startswith(("http", "deleted", "removed", "nan"))]
    return texto

def remove_stopwords(text):
    doc = nlp(text)
    tokens = [token.text for token in doc if token.text.lower() not in stop_words]
    return ' '.join(tokens)

def lemmatize_text(text):
    doc = nlp(text)
    lemmatized_tokens = [token.lemma_ for token in doc]
    return ' '.join(lemmatized_tokens)

def crear_label(label):
    if label == "ansiedad":
        return 1
    elif label == "Depresion":
        return 2
    else:
        return 0

# Preprocesar el contenido
df_merged['processed_content'] = preprocess_text(df_merged['content'])

# Remover valores NaN
df_merged.dropna(subset=['processed_content'], inplace=True)

# Remover stopwords y lematizar
df_merged['processed_content'] = df_merged['processed_content'].apply(remove_stopwords)
df_merged['processed_content'] = df_merged['processed_content'].apply(lemmatize_text)

# Eliminar duplicados en contenido procesado
df_merged = df_merged.drop_duplicates(subset='processed_content', keep='first')

# Convertir etiquetas a valores binarios
df_merged['target'] = df_merged['subreddit'].apply(crear_label)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(df_merged['processed_content'], df_merged['target'], test_size=0.2, random_state=42)


### Pipeline

In [6]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Configurar el pipeline con los modelos y vectorizadores
pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', LogisticRegression()) # en este caso solo probaremos un modelo
])

# Definir los parámetros para la búsqueda en cuadrícula
param_grid = {
    'vectorizer': [CountVectorizer(), TfidfVectorizer()], # ejercicio con dos vectorizaciones diferentes
    'classifier__C': [0.1, 1, 10] # ajuste de hiperparámetros para LogisticRegression()
}

# Configurar y ejecutar GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Mejor combinación de hiperparámetros
print("Mejores parámetros encontrados por GridSearch:")
print(grid_search.best_params_)

# Evaluación del mejor modelo en el conjunto de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Calcular la precisión
precision = accuracy_score(y_test, y_pred)
print(f"Precisión del mejor modelo: {precision:.4f}")

# Informe de clasificación
print("Informe de Clasificación del mejor modelo:")
print(classification_report(y_test, y_pred))

# Matriz de confusión
print("Matriz de Confusión del mejor modelo:")
print(confusion_matrix(y_test, y_pred))

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Mejores parámetros encontrados por GridSearch:
{'classifier__C': 1, 'vectorizer': TfidfVectorizer()}
Precisión del mejor modelo: 0.7507
Informe de Clasificación del mejor modelo:
              precision    recall  f1-score   support

           0       0.72      0.82      0.77       203
           1       0.81      0.72      0.76       242
           2       0.73      0.72      0.73       261

    accuracy                           0.75       706
   macro avg       0.75      0.76      0.75       706
weighted avg       0.75      0.75      0.75       706

Matriz de Confusión del mejor modelo:
[[167   7  29]
 [ 26 174  42]
 [ 38  34 189]]


### Revisión de textos pertenecientes a r/espanol mal clasificados

In [7]:
# Crear un DataFrame con las predicciones y las verdaderas etiquetas
results_df = pd.DataFrame({
    'Original Text': X_test,
    'True Label': y_test,
    'Predicted Label': y_pred
})

# Filtrar los textos que deberían ser 0 pero fueron clasificados como 1 o 2
wrong_classifications = results_df[(results_df['True Label'] == 0) & (results_df['Predicted Label'] != 0)]

In [8]:
print("Caso perteneciente a r/espanol clasificado como r/depresion:")
print(wrong_classifications.loc[4201, "Original Text"])

Caso perteneciente a r/espanol clasificado como r/depresion:
callado introvertido tiempo ir dañar forma q vuelto adicto sustacia alcohol tratar encajar mala influencia destrui valor seguir amigo constantemente deprimido
